In [1]:
import os, sys
import cv2
import numpy as np
from yolo_utils import *
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f'Running on Python {sys.version}, Tensorflow {tf.__version__}, OpenCV {cv2.__version__}')

Running on Python 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)], Tensorflow 2.5.0, OpenCV 4.5.2


In [2]:
print('Loading models...')
DNN = "TF"  # Single Shot Multi-box Detector with ResNet-10 (SSD), TF for 8 bit mobile, or YOLO V3
assert DNN in ['TF', 'CAFFE', 'YOLO'], f'invalid DNN! Got: {DNN}'
if DNN == "CAFFE":
    modelFile = "face_detector/res10_300x300_ssd_iter_140000_fp16.caffemodel"
    configFile = "face_detector/deploy.prototxt"
    face_detector = cv2.dnn.readNetFromCaffe(configFile, modelFile)
elif DNN == 'TF':
    modelFile = "face_detector/opencv_face_detector_uint8.pb"
    configFile = "face_detector/opencv_face_detector.pbtxt"
    face_detector = cv2.dnn.readNetFromTensorflow(modelFile, configFile)
else:
    configFile = "face_detector/yolov3.cfg"
    modelFile = "face_detector/yolov3-wider_16000.weights"
    face_detector = cv2.dnn.readNetFromDarknet(configFile, modelFile)

print(f'Face detection model ({DNN}) loaded. Loading face mask model...')

Loading models...
Face detection model (TF) loaded. Loading face mask model...


In [3]:
model = load_model('MobileNetV2 Full mask')
model_img_height, model_img_width = model.input_shape[1:3]  # The dimensions that the model is trained on
model.summary()
print('All models loaded.')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 1, 1280)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               163968    
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0     

In [4]:
image = cv2.imread('test/1.jpg')
img_height, img_width = image.shape[:2]
# TODO: NMS
if DNN == 'YOLO':  # TODO: not working properly, spams predictions
    blob = cv2.dnn.blobFromImage(image, 1 / 255, (IMG_WIDTH, IMG_HEIGHT), [0, 0, 0], 1, crop=False)
    face_detector.setInput(blob)
    detections = face_detector.forward(get_outputs_names(face_detector))
    faces = post_process(image, detections, CONF_THRESHOLD, NMS_THRESHOLD)
else:
    blob = cv2.dnn.blobFromImage(image, 1.0)  # no cropping as face detector sucks at cropped pics
    face_detector.setInput(blob)
    detections = face_detector.forward()
    for i in range(0, detections.shape[2]):  # loop through each face detected
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # face detector confidence level
            # compute the coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([img_width, img_height, img_width, img_height])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(img_width - 1, endX), min(img_height - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel resize it to model dimensions, and preprocess it
            face = image[startY:endY, startX:endX]
            face = cv2.resize(cv2.cvtColor(face, cv2.COLOR_BGR2RGB), (model_img_height, model_img_width))
            face = np.expand_dims(preprocess_input(img_to_array(face)), axis=0)

            # pass the face through the model to determine if the face has a mask or not
            (noMask, Mask) = tf.nn.softmax(model.predict(face)[0])  # softmax here as model does not have due to using logit for loss

            # determine the class label and color we'll use to draw the bounding box and text
            label = "Mask On" if Mask > noMask else "No Mask"
            color = (0, 255*float(max(Mask, noMask)), 255*(1-float(max(Mask, noMask)))) if label == "Mask On" else (0, 255*(1-float(max(Mask, noMask))), 255*float(max(Mask, noMask)))
            label = f"{label}: {max(Mask, noMask) * 100:.3f}%"  # confidence for mask detector

            # display the label and bounding box rectangle on the output frame
            cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
cv2.imshow("Result", image)
cv2.waitKey(0)  # ms to wait before closing the window, 0 is infinite
cv2.destroyAllWindows()

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-kuwfz3h3\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
